## Setup

### RSA Functions

In [1]:
function find_inv(x, modp)
    ~, ~, x_inv = gcdx(modp, x);  # NB x_inv is not guaranteed positive
    x_inv = mod(x_inv, modp);   # This ensures that x_inv is strictly positive
    return x_inv;
end;

function find_coprime(x, lowerbound = 1)
    for i::Int = lowerbound+1:typemax(Int)
        if gcd(i, x) == 1
            return i;
        end;
    end;
end;

function RSA_gen_keys(p1, p2)
    n = p1 * p2;
    t = (p1 - 1) * (p2 - 1);
    e = find_coprime(t, 10000);
    d = find_inv(e, t)
    public_key = (n, e);
    private_key = d;
    return public_key, private_key;
end;

function RSA_Encrypt(message, public_key)
    n, e = public_key;
    return powermod(message, e, n);
end;

function RSA_Decrypt(cipher, public_key, private_key)
    n, e = public_key;
    return powermod(cipher, private_key, n);
end;

### Key Generation

In [2]:
@show p1 = 70783;
@show p2 = 102059;

@show public_key, private_key = RSA_gen_keys(p1, p2);

p1 = 70783 = 70783
p2 = 102059 = 102059
(public_key, private_key) = RSA_gen_keys(p1, p2) = ((7224042197, 10001), 1381788029)


## Examples

### Arithmetic

#### Basic Multiplication

In [3]:
message = 2
n, e = public_key;
ciphertext = RSA_Encrypt(message, public_key);
factor = RSA_Encrypt(2, public_key);
decoded = RSA_Decrypt(mod(ciphertext * factor, n), public_key, private_key);
@show decoded


decoded = 4


4

#### Array Product

In [ ]:
encrypt = x -> RSA_Encrypt(x, public_key);

multiply = (x, y) -> mod(Int128(x) * Int128(y), n)

code = map(encrypt, [10, 20, 30, 40])

code_product = reduce(multiply, code)

product = RSA_Decrypt(code_product, public_key, private_key);

@show product

In [ ]:
gcdx(5, 25)

#### Find Square Root

In [ ]:
encrypt = x -> RSA_Encrypt(x, public_key);
multiply = (x, y) -> mod(Int128(x) * Int128(y), n)
power = (x, y) -> mod(reduce(multiply, ones(Int, y) * x), n);

message = 9;

@show code = encrypt(message)
@show mod(encrypt(3), n)
@show mod(encrypt(3)^2, n)

# c, a, b = gcdx(code, 2)
# @show mod(a, n), mod(b,n)
# @show mod(b,n) < code
# @show encrypt(3) < mod(b,n)

_, _, limit = gcdx(code, 2)
limit = mod(limit, n)

code_root = 0;
for i = 2:limit
#     if powermod(i, 2, n) == code
    if gcd(i, code) == i
        code_root = i
        break;
    end    
end

@show code_root
root = RSA_Decrypt(code_root, public_key, private_key);

@show root

### Find Geometric Mean

In [ ]:
encrypt = x -> RSA_Encrypt(x, public_key);
multiply = (x, y) -> mod(Int128(x) * Int128(y), n)
power = (x, y) -> mod(reduce(multiply, ones(Int, y) * x), n);

values = [10, 20, 30, 40]
values = [2, 2, 2, 2]

code = map(encrypt, values)

code_product = reduce(multiply, code)

num = length(code)

code_root = 0
for i = 2:code_product
    if powermod(i, num, n) == code_product
        code_root = i
        break;
    end    
end

@show code_root
avg = RSA_Decrypt(code_root, public_key, private_key);

@show avg

### Logic

#### AND

The logical AND operation is analoguos to multiplication


In [ ]:
message = 1
n, e = public_key;
ciphertext = RSA_Encrypt(message, public_key);
@show ciphertext
state = RSA_Encrypt(0, public_key);
@show state
decoded = RSA_Decrypt(mod(ciphertext * state, n), public_key, private_key);
@show decoded

Of course, due to our basic implementation of RSA Encryption, encrypting a 1 or a 0 is pretty useless

Their identitiy properties make it a obvious to decode encryption.

$0^n=0$

$1^n=1$

Instead, we could consider adding randomness to obviouscate it

In this new representation, we represent 0 as any even number, and 1 with any odd number.

Bob calculatestheir producct, oblivious to their encrypted meaning (without significant work which is not useful int he future)

When Alice recieves and decrypts the product, she checks if the product is odd to see if it is true.


In [4]:
using Random

message = 1
noise = rand(2:n-2)
randomisedMessage = noise + mod(noise, 2) + message;

ciphertext = RSA_Encrypt(randomisedMessage, public_key);
@show ciphertext

state = 0;
noise = rand(2:n-2);
randomisedState = noise + mod(noise, 2) + state;

cipherState = RSA_Encrypt(randomisedState, public_key);
@show cipherState

decoded = RSA_Decrypt(mod(ciphertext * cipherState, n), public_key, private_key);
@show decoded
@show mod(decoded, 2);

ciphertext = 1320856871
cipherState = 4024441177
decoded = 5760725865
mod(decoded, 2) = 1


In [14]:
#  Potential issue

@show h = RSA_Encrypt(2, public_key);
@show l = RSA_Encrypt(2, public_key);
@show mod(randomisedMessage, 2)
@show mod(randomisedState, 2)

h = RSA_Encrypt(2, public_key) = 2227520075
l = RSA_Encrypt(2, public_key) = 2227520075
mod(randomisedMessage, 1) = 0
mod(randomisedState, 2) = 0


0

#### NOT
The not operation in this randomised case needs to be a multiplication which takes use from odd to even, and vica versa.

even x even = even

even x odd = even

ood x odd = odd

even / even = odd or even

even / odd = even


#### OR
Normally the OR operation would require something analoguous to an add, but if we apply de Morgans theorem
    
NOT()

### Game of Life

In [ ]:
function encode(table)
    for element
        rbase = randint(n);
        reven = r + mod(r, 2)
        randomisedElement = reven + element;
    end
end



function play(table)
    for i
        for j
            # a, b, c, d, e, f, g, h
        end
    end
end









